## OpenAI (ChatGPT)

Beispiel Beispiel für das Aufbereiten eines Repositories für OpenAI

Zuerst muss der Open AI API Key erstellt und ein Guthaben bereitgestellt werden:
* [API Key erstellen](https://platform.openai.com/account/api-keys)
* [Guthaben bereitstellen](https://platform.openai.com/settings/organization/billing/overview)

Mittels API Key erstellen wir den `client` welcher die Verbindung zum OpenAI API herstellt.


In [ ]:
import os
from openai import OpenAI
import json
from nbconvert import MarkdownExporter
from nbconvert.preprocessors import ExtractOutputPreprocessor
from IPython.display import Markdown, display
from datetime import datetime

key = ''
client = OpenAI(api_key=key)

***

### Vector Store 

Für das Speichern der Daten vom Repository verwenden wir einen Vector Store.

Die Arbeiten sind grob wie folgt:
- Klonen des Repositories
- Aufbereiten der Dateien in einem Vector Store


In [ ]:
%%bash
git clone https://github.com/mc-b/lernmaas.git

In [ ]:
# 1. Vektor-Store erstellen
repository_slug = "lernmaas"

try:
    vector_store = client.vector_stores.create(name=repository_slug)
    vector_store_id = vector_store.id
    print(f"Vektor-Store mit ID '{vector_store_id}' erstellt.")
except Exception as e:
    print(f"Fehler beim Erstellen des Vektor-Stores: {e}")
    exit()

# 2. Dateien in Verzeichnis suchen und hochladen
directory = repository_slug
supported_extensions = [".md", ".pdf", ".yaml", ".yml", ".ipynb", ".sh"]

for root, _, files in os.walk(directory):
    for filename in files:
        filepath = os.path.join(root, filename)
        file_extension = os.path.splitext(filename)[1]

        if file_extension in supported_extensions:
            try:
                if file_extension == ".ipynb":
                    # Notebook in Markdown umwandeln
                    exporter = MarkdownExporter()
                    exporter.register_preprocessor(
                        ExtractOutputPreprocessor(output_dir='output_files'),
                        enabled=True
                    )
                    body, resources = exporter.from_filename(filepath)
                    markdown_filepath = os.path.splitext(filepath)[0] + ".md"
                    with open(markdown_filepath, "w", encoding="utf-8") as md_file:
                        md_file.write(body)

                    with open(markdown_filepath, "rb") as file:
                        uploaded_file = client.files.create(file=file, purpose='assistants')
                        client.vector_stores.files.create(
                            vector_store_id=vector_store_id,
                            file_id=uploaded_file.id
                        )
                        print(f"Notebook '{filepath}' konvertiert '{markdown_filepath}'.")

                    os.remove(markdown_filepath)

                elif file_extension in [".yaml", ".yml"]:
                    # YAML-Inhalt in Markdown-Codeblock schreiben
                    with open(filepath, "r", encoding="utf-8") as f:
                        yaml_content = f.read()

                    markdown_content = f"# Datei {filename}\n\n```yaml\n{yaml_content}\n```"
                    markdown_filepath = os.path.splitext(filepath)[0] + ".md"

                    with open(markdown_filepath, "w", encoding="utf-8") as md_file:
                        md_file.write(markdown_content)

                    with open(markdown_filepath, "rb") as file:
                        uploaded_file = client.files.create(file=file, purpose='assistants')
                        client.vector_stores.files.create(
                            vector_store_id=vector_store_id,
                            file_id=uploaded_file.id
                        )
                        print(f"YAML '{filepath}' in Markdown verpackt: '{markdown_filepath}'.")

                    os.remove(markdown_filepath)
                    
                elif file_extension in [".sh"]:
                    # Script-Inhalt in Markdown-Codeblock schreiben
                    with open(filepath, "r", encoding="utf-8") as f:
                        yaml_content = f.read()

                    markdown_content = f"# Datei {filename}\n\n```\n{yaml_content}\n```"
                    markdown_filepath = os.path.splitext(filepath)[0] + ".md"

                    with open(markdown_filepath, "w", encoding="utf-8") as md_file:
                        md_file.write(markdown_content)

                    with open(markdown_filepath, "rb") as file:
                        uploaded_file = client.files.create(file=file, purpose='assistants')
                        client.vector_stores.files.create(
                            vector_store_id=vector_store_id,
                            file_id=uploaded_file.id
                        )
                        print(f"'{filepath}' in Markdown verpackt: '{markdown_filepath}'.")

                    os.remove(markdown_filepath)                    
                    

                else:
                    with open(filepath, "rb") as file:
                        uploaded_file = client.files.create(file=file, purpose='assistants')
                        client.vector_stores.files.create(
                            vector_store_id=vector_store_id,
                            file_id=uploaded_file.id
                        )
                        print(f"Datei '{filepath}' ok")

            except Exception as e:
                print(f"Fehler beim Verarbeiten der Datei '{filepath}': {e}")

print("✅ Fertig mit Hochladen und Zuordnen zum Vector Store.")


***

### Abfrage

Mittels zwei Hilfsfunktionen fragen wir unser Wissen ab:
- `get_vector_store_id_by_name` - Gibt die ID eines Vector Stores anhand seines Namens zurück.
- `query_vector_store` - Führt eine semantische Suche im Vector Store durch.

In [ ]:
def get_vector_store_id_by_name(target_name):
    try:
        after = None
        target_name_lower = target_name.lower()

        while True:
            # Abrufen mit oder ohne Cursor
            if after:
                response = client.vector_stores.list(limit=100, after=after)
            else:
                response = client.vector_stores.list(limit=100)

            for vs in response.data:
                if vs.name.lower() == target_name_lower:
                    return vs.id

            # Wenn weniger als 100 Ergebnisse => keine weitere Seite
            if len(response.data) < 100:
                break

            # Cursor für nächste Seite setzen
            after = response.data[-1].id

        print(f"Kein Vector Store mit dem Namen '{target_name}' gefunden.")
        return None

    except Exception as e:
        print(f"Fehler beim Abrufen der Vector Stores: {e}")
        return None

def query_vector_store(vector_store_id, query, top_k=3):
    try:
        response = client.responses.create(
            model="gpt-4o-mini",
            input=query,
            tools=[{
                "type": "file_search",
                "vector_store_ids": [vector_store_id]
            }]
        )
        return response
    except Exception as e:
        print(f"Fehler bei der Suche im Vector Store: {e}")
        return None
    

Dann folgt die eigentliche Abfrage:
- `vector_store_name`- Name des Vector Stores
- `user_input`- die eigentliche Frage

In [ ]:
vector_store_name = "lernmaas"
user_input = "was ist der Zweck dieses Repositories?"

# Führe eine Suche im Vector Store durch
vector_store_id = get_vector_store_id_by_name(vector_store_name)
response = query_vector_store(vector_store_id, user_input)

# Extrahiere den Markdown-Text aus der Antwort
if response:
    # response.output enthält eine Liste mit Nachrichten
    for item in response.output:
        if item.type == "message" and hasattr(item, "content"):
            for content_item in item.content:
                if content_item.type == "output_text" and hasattr(content_item, "text"):
                    markdown_text = content_item.text
                    display(Markdown(markdown_text))
                    


- - -

### Aufräumen 

löschen des Vector Stores!

In [ ]:
# Führe eine Suche im Vector Store durch
id = get_vector_store_id_by_name(vector_store_name)
# Lösche den Vector Store
client.vector_stores.delete(vector_store_id=id)


**Löschen von Doppelten Vectors Stores**

In [ ]:
try:
    all_vector_stores = []
    after = None

    while True:
        if after:
            response = client.vector_stores.list(limit=100, after=after)
        else:
            response = client.vector_stores.list(limit=100)

        data = response.data
        all_vector_stores.extend(data)

        if len(data) < 100:
            break

        after = data[-1].id

    if all_vector_stores:
        print("Vorhandene Vector Stores:")

        # Dictionary zur Erkennung doppelter Namen
        seen_names = {}
        duplicates = []

        for vs in all_vector_stores:
            if vs.name in seen_names:
                duplicates.append(vs)  # Duplikat merken
            else:
                seen_names[vs.name] = vs

        # Ausgabe aller Vector Stores
        for vs in all_vector_stores:
            print(f"- ID: {vs.id}, Name: {vs.name}, Created At: {vs.created_at}")
            print(f"  - Dokumente im Vector Store: {vs.file_counts}")

        print(f"\n📊 Gesamtanzahl Vector Stores: {len(all_vector_stores)}")
        print(f"🔍 Gefundene doppelte Vector Stores: {len(duplicates)}")

        # Duplikate löschen
        for dup in duplicates:
            try:
                client.vector_stores.delete(dup.id)
                print(f"❌ Gelöscht: ID {dup.id}, Name: {dup.name}")
            except Exception as del_err:
                print(f"⚠️ Fehler beim Löschen von {dup.id}: {del_err}")

    else:
        print("Es wurden keine Vector Stores gefunden.")

except Exception as e:
    print(f"Fehler beim Auflisten der Vector Stores: {e}")
